# python 简洁之美

## 一行代码交换a,b

In [1]:
a, b = 1, 4

a, b = b, a
a,b

(4, 1)

## 一行代码反转列表

In [2]:
[1,2,3][::-1] 

[3, 2, 1]

## 一行代码合并两个字典

In [3]:
{**{'a':1,'b':2}, **{'c':3}}

{'a': 1, 'b': 2, 'c': 3}

## 一行代码列表去重

In [4]:
set([1,2,2,3,3,3])

{1, 2, 3}

## 一行代码求多个列表中的最大值

In [5]:
max(max([ [1,2,3], [5,1], [4] ], key=lambda v: max(v))) 

5

## 一行代码生成逆序序列

In [6]:
list(range(10,-1,-1))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

# python概览

## 任务

假设由于每种原因你现在需要去记录人们的信息。你想写个程序来这些人的详细信息。也就是说你想把这些记录存在数据库中，以便在电脑中永久存储这些信息。  

你可以找到现成的数据库管理软件来管理对应的数据库。但是你可以亲自编写这样一个程序，可以完全控制它的运行方式，于是就这样入python的坑.........

## 第一步 表示记录

如果我们想在数据库中存储记录，就要确定以什么样的数据类型来存储。通常使用列表和字典这样的内建对象类型就够用了。

### 试用 List

#### 新建记录

In [7]:
bob = ["Bob Smith", 42, 20000, "software"]
sue = ["Sue James", 30, 30000, "hardware"]

#### 通过对列表的访问，处理可以容易地处理记录

In [8]:
#　Bob　姓啥？
bob[0].split()[-1]

'Smith'

In [9]:
# 给 sue 涨薪 25%
sue[2] *= 1.25
sue

['Sue James', 30, 37500.0, 'hardware']

#### 数据库列表

为了把 Bob 和 Sue 存储在一起，我们可以简单地把他们放入一个新的列表里，形成一个简单的数据库

In [10]:
people = [bob, sue]
# 查看一个这个数据库的内容
for person in people:
    print(person)

['Bob Smith', 42, 20000, 'software']
['Sue James', 30, 37500.0, 'hardware']


通过操作数据库来处理记录

In [11]:
# 给每人涨薪 20%
for person in people:
    print(person[0].split()[-1])  # 打印姓氏
    person[2] *= 1.20   # 涨薪水

Smith
James


In [12]:
# 检查一下有没有涨薪水
for person in people:
    print(person[2])

24000.0
45000.0


#### 使用python迭代工具从记录中获取各种值。例如获取列表解析、map和生成器表达式

In [13]:
# 获取员工的薪水列表
# 列表解析
pays = [person[2] for person in people]
pays

[24000.0, 45000.0]

In [14]:
# map
pays = map((lambda x: x[2]), people)
list(pays)

[24000.0, 45000.0]

In [15]:
# # 查看公司应付员工薪水总额
# 生成器表达式， sum为内建函数
sum(person[2] for person in people)

69000.0

#### Field标签

通过对列表的位置来访问字段导致我们一直需要记住列表每个位置的含义，这样编码麻烦而且代码可读性也差，最好把字段名和字段的值关联起来，也就是咱们给列表的每个位置起个名字。

In [16]:
NAME, AGE, PAY, JOB = range(4)
kobe = ["Kobe Bryant", 60, 60000, "spokesperson"]

In [17]:
# 查看姓名
kobe[NAME]

'Kobe Bryant'

In [18]:
# 查看职位
JOB, kobe[JOB]  

(3, 'spokesperson')

这样兄弟萌就解决了可读性问题，但是我们发现数据记录和字段名并非是关联在一起的，这样随着时间的前进，后来的编码人员对数据结构进行变化时，如果忘了对字段变量做相应的维护的话，它们将会失去同步。  

而且字段名是独立的变量，没有从记录到其字段名的直接映射。  

我们可以尝试利用元组的列表来解决这个问题，其中每个元组保存字段名和值，更加理想的是，使用列表的列表允许进行更新的，（元组是不可变的）

In [19]:
bob = [['name', "Bob Smith"], ["age", 42], ["pay", 30000]]
sue = [['name', "Sue James"], ["age", 46], ["pay", 40000]]
people = [bob, sue]

但是这样我们依然需要通过列表的index值来获取数据。

In [20]:
# 收集人员姓名
[person[0][1] for person in people]

['Bob Smith', 'Sue James']

实际上仅仅是增加了一层额外的位置索引。更好的办法是，可以在循环中检查字段名字来找到我们想要的属性

In [21]:
for person in people:
    for (name, value) in person:
        if name == "name" :
            print(value)

Bob Smith
Sue James


我们可以抽取其中的逻辑写个函数接口来做项工作，以后调用函数进行复用就可以了

In [22]:
def field(record, label):
    """
    record: 记录变量名
    label： 字段名
    """
    for(fname, fvalue) in record:
        if fname == label:  # 根据名字来寻找字段名
            return fvalue

In [23]:
# 获取姓名
field(bob, "name")

'Bob Smith'

In [24]:
# 打印所有年龄
for rec in people:
    print(field(rec, "age"))   #简化了代码，提升了可读性

42
46


如果继续探索下去，最终会实现一套将字段名映射到字段值的完善的记录接口函数。

#### 持久存储问题

列表的确可以用作我们的人员数据库，而且的确它也满足一些小程序的需要。但是他是在内存中，一旦python的进程退出，这些数据就会从内存中消失。

### 试用字典

#### 新建字典数据表示

虽然列表的记录表示方式能够使用，但是我们知道内建的字典对象在关联属性的名字和值方面会更高效和方便

In [25]:
bob = {"name":"Bob Smith", "age":42, "pay":30000, "job":"dev"}
sue = {"name":"sue James", "age":46, "pay":40000, "job":"mng"}

In [26]:
# 相比list我们不再需要记住index值偏移量对应的含义
bob["name"]

'Bob Smith'

In [27]:
# 涨薪  
print(sue["pay"])
sue["pay"] *= 1.10
print(sue["pay"])

# 代码可读性也大大地好
# 字段名和值之间的映射关系也稳定，也不用担心它们之间会失联

40000
44000.0


#### 其他建立字典的方法

可以使用关键词参数和类型构造函数，主要所有的键是字符串

In [28]:
bob = dict(name="Bob Smith", age=42, pay=30000, job="dev")
sue = dict(name="Sue Jones", age=45, pay=40000, job="hdw")
bob

{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}

或者一次一个字段地填写（字典的键是伪随机排列的呦）

In [29]:
wade = {}

In [30]:
wade["name"] = "D.Wade"
wade

{'name': 'D.Wade'}

In [31]:
wade["age"] = "40"
wade

{'name': 'D.Wade', 'age': '40'}

或者用zip函数将 名/值 两个列表链在一起

In [32]:
keys = ["name", "age", "pay", "job"]
values = ["L.J", "35", "35000000$", "basketball player"] 

# 1
list(zip(keys, values))

[('name', 'L.J'),
 ('age', '35'),
 ('pay', '35000000$'),
 ('job', 'basketball player')]

In [33]:
# 2
dict(zip(keys, values))

{'name': 'L.J', 'age': '35', 'pay': '35000000$', 'job': 'basketball player'}

甚至可以通过一个键序列和所有键的可选初始值来创造字典（便于初始化空字典）

In [34]:
# 先把结构创建出来，然后可以在慢慢填入
fields = {"name", "age", "job"}
record = dict.fromkeys(fields, "?")
record

{'name': '?', 'job': '?', 'age': '?'}

#### 字典列表数据库

不管怎样实现字典，仍然需要把基于字典的数据记录会聚在一起，形成数据库。我们依然使用列表

In [35]:
bob

{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}

In [36]:
sue

{'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}

In [37]:
people = [bob, sue]
for person in people:
    print(person["name"], person["pay"])

Bob Smith 30000
Sue Jones 40000


对数据库可以使用迭代工具

In [38]:
# 人员有哪些
names = [person["name"] for person in people]
names

['Bob Smith', 'Sue Jones']

In [39]:
# map
list(map((lambda x: x["name"]), people))

['Bob Smith', 'Sue Jones']

In [40]:
# 总共应付人员薪水
sum(person["pay"] for person in people)

70000

In [41]:
db = {}                 # 数据库
db["bob"] = bob         # 引用字典的字典
db["sue"] = sue

由于字典是python对象，所以数据库中的记录可以用python语法进行增删改查

In [42]:
for person in people:
    person["pay"] *= 1.10  # 为所有人员加薪

#### 嵌套结构

In [43]:
# 现在抽取姓氏时
for person in people:
    print(person["name"].split()[-1])

Smith
Jones


通过再进一步结构化我们的记录，可以避免上述列子中这样的抽取姓氏的代码

In [44]:
# 将字典、列表、元组嵌套在另一个字典中
bob2 = {"name":{"first":"bob", "last":"Smith"},
        "age":42,
        "job":["software", "dancer"],
        "pay":(30000, 20000),
       }

In [45]:
# 获取全名
bob2["name"]

{'first': 'bob', 'last': 'Smith'}

In [46]:
# 获取bob的姓
bob2["name"]["last"]

'Smith'

In [47]:
# bob 的所有工作
for job in bob2["job"] : print(job) 

software
dancer


In [48]:
# bob 又增加了一个工作
bob2["job"].append("lol player")

In [49]:
bob2["job"]

['software', 'dancer', 'lol player']

#### 字典的字典

我们对基于字典的人员数据库做最后一次改进，可以从字典中多挖掘一些字典的好处。  

我们使用字典表示数据库本身，

In [50]:
bob = dict(name="Bob Smith", age=42, pay=30000, job="dev")
sue = dict(name="Sue Jones", age=45, pay=40000, job="hdw")

In [51]:
db = {} # 建立一个数据库

In [52]:
# 引用字典的字典
db["bob"] = bob 
db["sue"] = sue

In [53]:
db["bob"]["name"]

'Bob Smith'

In [54]:
db["sue"]["pay"] * 1.1  # 涨薪

44000.0

In [55]:
# 这样的结构可以直接访问一条记录，而不必循环中寻找到它。
db #查看数据库

{'bob': {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'},
 'sue': {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}}

In [56]:
# python的pprint可以让输出更易于阅读
import pprint
pprint.pprint(db)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000}}


如果仍然需要逐条记录地访问数据库，可以使用for循环和字典的迭代器

In [57]:
# 迭代起
x = [db[key]["name"] for key in db]
y = [rec["name"] for rec in db.values()]
print(x, y)

['Bob Smith', 'Sue Jones'] ['Bob Smith', 'Sue Jones']


#### 数据库增删改查

In [58]:
#　数据库的记录总数
len(db)

2

In [59]:
# 增 - 只要把新的记录赋给一个新的键即可
db["tom"] = dict(name="Tom", age=50, job=None, pay=0)
pprint.pprint(db)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000},
 'tom': {'age': 50, 'job': None, 'name': 'Tom', 'pay': 0}}


In [60]:
# 查看数据库有哪些人员记录
list(db.keys())

['bob', 'sue', 'tom']

In [61]:
# 类似 SQL 的条件查询
[rec["name"] for rec in db.values() if rec["age"] >= 45] 

['Sue Jones', 'Tom']

#### 持久存储问题

虽然我们的数据库仍然是内存的临时对象，当python进程死亡时，这些数据也随之消失，但是这种字典的结构与永久性对象的系统结构非常切合--shelve(英文语法上应该是shelf，但是python模块命名和术语用的shelve)

## 第二步  持久化存储记录

由于前面的所有对象是在内存中的都是暂时的，所以为了人员信息持久化，我们需要将对象保存在某种类型的文件中。

### 使用格式化文件

程序运行时保存数据的一种方法是把所有数据以格式化的方法写入一个简单的文本文件中。只要保存和载入的工具在所选格式上达成一致就可以

#### 生成测试数据的接口（可设计为脚本模块，命令行运行）

In [62]:
# 可以单独制作成一个py文件作为脚本，通过__main__入口调用
def initdata():
    #　记录       
    bob = dict(name="Bob Smith", age=42, pay=30000, job="dev")
    sue = dict(name="Sue Jones", age=45, pay=40000, job="hdw")
    tom = dict(name="Tom Sun", age=48, pay=0, job="")

    # db
    db = {}
    db["bob"] = bob
    db["sue"] = sue
    db["tom"] = tom
    
    return db

# 生成
initdata() 

{'bob': {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'},
 'sue': {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'},
 'tom': {'name': 'Tom Sun', 'age': 48, 'pay': 0, 'job': ''}}

#### 数据格式化脚本

建立一些 标识变量

In [63]:
dbfilename = "people-file"   # 文件名
ENDDB = "enddb."     # 数据库分割标识        
ENDREC = "endrec."   # 记录间分割标识
RECSEP = "=>"        # 记录的 键/值 映射标识

数据格式化保存接口

In [64]:
def storeDb(db, dbfilename=dbfilename):
    "将数据格式化保存为不同文件"
    dbfile = open(dbfilename, "w")
    for key in db:
        print(key, file=dbfile)  # dbfile.write(key + "\n") 同效
        for (name, value)in db[key].items():
            print(name + RECSEP + repr(value), file=dbfile)  #　repr将其当作代码执行
        print(ENDREC, file=dbfile)
    print(ENDDB, file=dbfile)
    dbfile.close()

In [65]:
# 将测试数据持久化存储
storeDb(initdata())  

In [66]:
# 浏览一下这个文件
for line in open("people-file"):
    print(line, end="")

bob
name=>'Bob Smith'
age=>42
pay=>30000
job=>'dev'
endrec.
sue
name=>'Sue Jones'
age=>45
pay=>40000
job=>'hdw'
endrec.
tom
name=>'Tom Sun'
age=>48
pay=>0
job=>''
endrec.
enddb.


从文件解析数据并重构数据库对象

In [67]:
def loadDb(dbfilename=dbfilename):
    "解析数据， 重构数据库"
    db = {}         # 数据库的空间
    rec = {}        # 记录的缓存空间
    recname = ""    # 数据库的  键
    with open(dbfilename, "r") as dbfile:
        for line in dbfile :
            line = line.strip("\n")        
            if line != ENDDB:  
                if line != ENDREC:
                    if RECSEP  in line : 
                        name, value = line.split(RECSEP)
                        rec[name] = eval(value)    # 往记录添加数据
                       
                        # print(recname)
                    else:
                        recname = line  #　记录名
                        db[recname] = rec
                    # print(rec)
                else:
                    rec = {}    # 一旦当前记录结束，缓存置空，
                    continue    # 跳过 endrec. 这一行继续执行
            else:
                return db

In [68]:
db = loadDb()  # 解析文件并构建数据库

对重构的数据库可以进行增删改查，并重新写入文件

In [69]:
db["tom"]["job"] = "mng"
db["tom"]["pay"] = 50000
db["tom"]

{'name': 'Tom Sun', 'age': 48, 'pay': 50000, 'job': 'mng'}

In [70]:
storeDb(db)  # 将更改厚的数据库写入文件
loadDb()     # 检查一下是否更改生效


{'bob': {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'},
 'sue': {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'},
 'tom': {'name': 'Tom Sun', 'age': 48, 'pay': 50000, 'job': 'mng'}}

到此为止，就算计算机关机都不会丢失数据了，因为我们已经将数据库对象格式化保存到硬盘里的文件里了，我们可以靠专用的脚本载出更改和存入这些对象了。这样我萌就算把对象持久化了。

###  使用 pickle文件

虽然我们格式化文本文件结构是可行的，但是他有些较大的局限性。  

首先为了获取一条记录或者更改一条记录就需要读取和写入整个数据库，这样会让程序越来复杂越来越慢。  

其次，我们格式化的方法为例，用 => 做分割符的办法，当文本内容里出现 => 这样的内容，就会发生冲突，导致这种格式化的结构失败。虽然我们可以采用XML的方式来结构化，但是创建XML和解析XML的工作是很烦银的 

如果有一种通用的工具，它只需一步就可以将任意类型的Python对象转化成可以保存在文件中的格式，而你不需要关心，文件的格式化结构，不必为了解析文件而费心，那就好了。这正是Pickle设计的初衷。

Pickle模块将内存中的Python对象转化成序列化的字节流，这是一种可以写入任何类似文件对象的字节串。  
Pickle模块也可以根据序列化的字节流重新构建原来内存中的对象。将文件转换成原来的那个对象。  

可以说 Pickle模块取代了专有的数据格式。

In [71]:
# 存
import pickle 

db = initdata() # 生成测试数据
dbfile = open("people-pickle", "wb")   # 用字节的方式进行写
pickle.dump(db, dbfile)  # 把 db 对象pickle进文件里
dbfile.close()

In [72]:
# 取
import pprint

with open("people_pickle", "rb") as dbfile:   # 用字节的方式进行读 
     db_ = pickle.load(dbfile)  # 读取           
     pprint.pprint(db_)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000},
 'tom': {'age': 48, 'job': 'mng', 'name': 'Tom Sun', 'pay': 50000}}


In [73]:
# 更新
db["sue"]["pay"] *= 1.10

# 再写入
dbfile = open("people-pickle", "wb")  # 用字节的方式进行写
pickle.dump(db, dbfile)  # 把 db 对象pickle进文件里
dbfile.close()

我们观察到当记录在内存中被修改后是将整个数据库重新写入文件，这和我们自己手动格式化文件的方式一样的，当数据库比较庞大时，这些操作会变得很慢。暂时保留这个问题

Pickle和它的数据格式也可以处理shelve和ZODB数据库，pickle的类实例为被存储的对象提供了数据和操作。  

实际上Pickle更加通用，只要对象具有文件兼容的接口，就可以通过pickle将内存中的Python对象转换成多种媒介存储。  

可以使用网络套接字（Socket）可以将Pickle过的Python对象通过网络进行传输，甚至作为SOAP和XML-RPC这样的大型协议的代替。

### 每条记录使用一个pickle文件

前面提到的潜在的缺点，当数据库非常大是变得很慢，慢的原因是更新操作一条记录就要将整个数据库重新读取存入。  

我们可以通过存储数据库中的一条记录到一个普通文件的方式来改进。以记录本身的键为文件名

1. 将每条记录存到对应的文件中

In [74]:
import pickle

for (key, record) in [("bob", bob), ("sue", sue)]:
    with open(key+".pkl", "wb") as recfile:
        pickle.dump(record, recfile)

2  使用glob模块获取整个数据库。它根据文件拓展名来获取这个目录下所有该拓展名的文件。如果需要载入单个记录，可以打开对应的文件并使用pickle反序列化。为了得到一条记录，我们只需要载入一个记录而不需要整个数据库

In [75]:
import glob

# 查看整个数据库
for filename in glob.glob("*.pkl"):    
    with open(filename, "rb") as recfile:
        record = pickle.load(recfile)
        print(filename, "=>\n",record)

#　查看单个记录
suefile = open("sue.pkl", "rb")
print("sue.pkl['name']", "=>\n", pickle.load(suefile)["name"])
suefile.close()

bob.pkl =>
 {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue.pkl =>
 {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}
sue.pkl['name'] =>
 Sue Jones


3  当更新数据库，从对应文件抓取一条记录，在内存中进行修改，然后重新写入它的pickle文件，不用为了更新某段记录而对整个数据库进行操作

In [76]:
suefile = open("sue.pkl", "rb")
sue = pickle.load(suefile)
suefile.close()

sue["pay"] *= 1.10  # 给sue涨薪10%
suefile = open("sue.pkl", "wb")
pickle.dump(sue, suefile)
suefile.close()

# 查看该记录是否更新成功
suefile = open("sue.pkl", "rb")
print("sue.pkl['pay']", "=>\n", pickle.load(suefile)["pay"])
suefile.close()

sue.pkl['pay'] =>
 44000.0


上述方法数据库的中的key变为了实际的文件名。也就是说文件系统成为了高层字典，通过文件名可以直接访问每条记录

### 使用shelves 
假设你的文件系统可以处理任意多需要的文件，将一条记录pickle成一个大文件。避免了每次更新和操作时都需要载入和存储整个数据库，如果我们用键来访问记录，Python表批准库提供了一中更高层次的工具：shelves

它自动地将对象pickle进和pickle出健访问文件系统。它们很像必须打开着的字典，在程序退出时进行持久化。shelves使用起来像是必须打开着的一个字典。 

1. 创建数据库

In [77]:
import shelve

db = shelve.open("people-shelve") 
db["bob"] = bob
db["sue"] = sue
db.close()

2. 访问数据库, 依旧是字典的字典

In [78]:
# 像字典一样通过键来访问
db = shelve.open("people-shelve")
for key in db:
    print(key, "=>\n", db[key]) 

bob =>
 {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue =>
 {'name': 'Sue Jones', 'age': 45, 'pay': 44000.0, 'job': 'hdw'}
tom =>
 {'name': 'Tom', 'age': 50, 'job': None, 'pay': 0}


3. 更新

In [79]:
# 需要加入的新数据
tom= dict(name="Tom", age=50, job=None, pay=0)

# 打开数据库
db = shelve.open("people-shelve") # 缺省的方式
# 更改记录
sue = db["sue"]   
sue["pay"] *= 1.1

# 手动写回的原因是缺省需求，可以在open中定义关闭shelve时写回，但是这么干会消耗内存使得关闭变慢
# 缺省：需要时创建，否则以可读可写方式打开 
db["sue"] = sue

# 添加新记录
db["tom"] = tom
db.close() # 关闭shelve、关闭缓存通道

# 可以将数据库操作单独制作成脚本文件，通过运行相应程序 自动对本地数据库操作并将变化状态持久化地保存在文件中，

## 第三步 走进OOP

面向对象的方式能使得实现类似操作时避免代码的重复编写，提高代码的复用，代码的调试和维护的难度也降低了。

### 构建类

实例化类的方式来实现记录数据库

In [80]:
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job

构建记录（类的实例）

In [81]:
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")
bob.name.split()[-1] # 获取姓氏

'Smith'

更新记录

In [82]:
# 涨薪
sue.pay *= 1.1
sue.pay

38500.0

这个类还不是一个数据库，将类的实例放入列表和字典，构成一个整体

In [83]:
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")

people = [bob, sue]  # 构成一个数据库列表
for person in people:
    print(person.name, person.pay)

Bob Smith 30000
Sue Jones 35000


查询

In [84]:
x = [(person.name, person.age, person.pay,  person.job) for person in people]
x

[('Bob Smith', 42, 30000, 'software'), ('Sue Jones', 45, 35000, 'hartware')]

SQL 风格查询

In [85]:
[rec.name for rec in people if rec.age >= 45]

['Sue Jones']

### 为类添加行为

现在我们们实例化类仅仅是得到数据。它使用对象的属性替代字典的键，但是并没有原来的那些操作方法，为了利用类的功能完成这些操作，我们为类添加一些行为。

In [86]:
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job
    
    # 捕获姓氏行为
    def get_LastName(self):
        return self.name.split()[-1]
    
    # 涨薪行为
    def giveRaise(self, percent=0.1):
        self.pay *= (1.0 + percent)

In [87]:
# 测试一下
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")

bob.get_LastName()

'Smith'

In [88]:
# 一直涨薪一直爽
print(sue.pay)
sue.giveRaise()
print(sue.pay)

35000
38500.0


### 继承  通过人员类型编写子类

构建管理层子类， 可以单独地对管理层进行一些定制化的行为，而不会影响到父类（所有人员）

In [89]:
# 比如管理层除了正常涨薪还有 额外百分比的奖金的话 哎
class Manager(Person):
    
    # 重写涨薪行为
    def giveRaise(self, percent=0.1, bonus=0.1) :
        self.pay *= (1.0 + percent +bonus)

In [90]:
tom = Manager(name="Tom Doe", age=50, pay=50000)
tom.get_LastName() # 父类的功能

'Doe'

汤姆因为是Manager类的实例，给他涨薪的时候，除了父类的10%还有自定义的奖金

In [91]:
print(tom.pay)
# 公司默认奖金10%，但是这次表现突出给奖金20%，加上基础所有人员涨薪10%，这样就是30%了哦
tom.giveRaise(bonus=.20) 
tom.pay


50000


65000.0

这样可以在父类中全局控制所有记录的共有属性，在子类中管理各个类型的特有行为，除了可维护性高，也不会在记录间相互干扰。

###  重构代码

####  增强方法

我们发现涨薪计算在两个地方重复出项了（在两个类中）。我们对子类的涨薪方法可以通过增强继承来实现，而不是完全替代父类方法（完全重写），从而达到去除代码中的冗余的效果。

In [92]:
class Manager(Person):
    
    def giveRaise(self, percent=0.1, bonus=0.1) :
        Person.giveRaise(self, percent + bonus)
# 1.调用父类的方法显式地传入self参数
# 2. 虽然重新定义了该方法，但我们调用的是通用的方法，可以降低冗余（业务逻辑只出现一次）
# 3.实践中易于启动父类的构造器，这样在父类中重新定义了percent参数设置时，子类中会启动父类构造器，自动生效。

####  格式化显示

为了获得更多面向对象的乐趣，咱们要为people类增加一些操作符重载。

In [93]:
# 这是还没重载时的输出，默认的只有地址信息，而且可读性也是挺差的
tom = Manager("Tom Jams", 60, 50000)
print(tom)

In [94]:
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job
    
    # 操作符重载
    def __str__(self):
        # 当打印 Person 实例时 python 会自动调用这个方法滴，默认的返回值信息少的很
        # 自定义返回值,对当对象需要作为整体打印时，比实例缺省的显示好得多
        return "<%s => %s: %s, %s>" % (self.__class__.__name__, self.name, self.job, self.pay)
    # 可以编写 __add__ 使得 + 表达式自动调用 giveraise方法，代价是代码可读性变差，不知到具体你 + 了啥？
    # 还有更多的操作符可以重载 __repr__、 __call__、 __iter__ 等等...
    
    
    def get_LastName(self):
        return self.name.split()[-1]
    
    def giveRaise(self, percent=0.1):
        self.pay *= (1.0 + percent)

class Manager(Person):
    
    def giveRaise(self, percent=0.1, bonus=0.1) :
        Person.giveRaise(self, percent + bonus)

In [95]:
# 测试一下打印时输出有啥改变， 呦！
tom = Manager("Tom Jams", 60, pay=6000, job="manager")
print(tom)

<Manager => Tom Jams: manager, 6000>


#### 自定义构造函数

我们又发现了一个问题：  tom = Manager("Tom Jams", 60, pay=6000, job="manager")  
这个类实例化的语句中命名参数： job="manager" 存在着冗余，想想看这是manger类的实例，那job这个命名参数有点废话的赶脚嘛  
比较合理的是为manager类提供一个显式的构造函数，自动填写job参数。

In [96]:
# 说干就干
class Manager(Person):
    
    def __init__(self, name, age, pay):
        Person.__init__(self, name, age, pay, job="Manager") # 这里调用父类的构造器，并自动填写job

    def giveRaise(self, percent=0.1, bonus=0.1) :
        Person.giveRaise(self, percent+bonus)

In [97]:
# 测试一下  格式化输出
bob = Person("Bob Smith", 44)    # 还没为bon分配工作
sue = Person("Sue Jones", 47, 40000, "Hardware") # 分配了硬件工程师（可以再写个硬件工程师子类）
tom = Manager(name="Tom Doe", age=50, pay=50000) # 分配到经理
for obj in (bob, sue, tom):
    obj.giveRaise(.10)
    print(obj)

<Person => Bob Smith: None, 0.0>
<Person => Sue Jones: Hardware, 44000.0>
<Manager => Tom Doe: Manager, 60000.0>


### 增加持久化

我们现在已经把记录和对其的操作封装进了类中。我们可以把她们存储在一个单记录的pickle文件中，而基于shelve的存储媒介也能很好的实现我们的想法。并且更容易编码。

In [98]:
# 创建记录（实例） 
bob = Person("Bob Smith", 44)
sue = Person("Sue Jones", 47, 40000, "Hardware")
tom = Manager(name="Tom Doe", age=50, pay=50000)

# 创建存储记录(实例)的shelve，也就是咱们的顶层数据库了
db = shelve.open("class-shelve")

# 然后将 记录 赋给新建的shelve文件的 键
db["bob"] = bob
db["sue"] = sue
db["tom"] = tom
db.close()

In [99]:
# 查看数据库
db = shelve.open("class-shelve")
for key in db:
    print(key, " =>\n ", db[key].name, db[key].pay)

bob  =>
  Bob Smith 0
sue  =>
  Sue Jones 40000
tom  =>
  Tom Doe 50000


In [100]:
#  对数据库中的实例进行操作
bob = db["bob"]
print(bob.get_LastName()) 


# 可以直接操作
# 不需要导入实例的相关类，pickle系统会自动定位相关类，只要类在模块搜索路径上。

Smith


由于类和他的实例在shelve中是被分别存储的，所以可以在外部通过修改类来改变实例被载入时的解释方法。  

可以对shleve数据库中存储的实例进行更改，因为这些对象都是持久化在shelve中的/。也可以通过python交互命令打开并查看shelve，尽管其存储具有长期性，但是shelve其实真正上只是一种包含了python对象的对象（它本身也是一个shelve实例）

### 其他数据库选项

shelve功能齐全，虽然它不是关系型数据库（我们存储的是对象，不是表），但是对于一些类型的程序这已经够用了

如果需要更多的功能，我们可以将这个应用移植到一些更强大的工具上。

# 数据结构和算法

# 数据结构的巧用

## 解压序列赋值给多个变量

### 问题 
现在有一个包含N 个元素的元组或者是序列，怎样将它里面的值解压后同时赋值
给N 个变量？

### 解决方案  
任何的序列(或者是可迭代对象) 可以通过一个简单的赋值语句解压并赋值给多个
变量。唯一的前提就是变量的数量必须跟序列元素的数量是一样的。

In [101]:
p = (4, 5)
data = [ 'ACME', 50, 91.1, (2012, 12, 21) ]
x, y  = p
name, shares, price, date = data
a, b, c, d, e = 'hello'

In [102]:
y

5

In [103]:
date

(2012, 12, 21)

In [104]:
d

'l'

### 讨论  
实际上，这种解压赋值可以用在任何可迭代对象上面，而不仅仅是列表或者元组。
包括字符串，文件对象，迭代器和生成器。

In [105]:
s = "Hello"
a, b, c, d, e = s

In [106]:
a

'H'

有时候，你可能只想解压一部分，丢弃其他的值。对于这种情况Python 并没有提
供特殊的语法。但是你可以使用任意变量名去占位，到时候丢掉这些变量就行了

In [107]:
data = [ 'ACME', 50, 91.1, (2012, 12, 21) ]
_, shares, price, _ = data  # python里想丢掉的一般用 _ 来占位

In [108]:
shares

50

In [109]:
price

91.1

# 字符串和文本

# 数字日期和时间

# 迭代器和生成器

# 文件与 IO

# 数据编码和处理

# 函数

# 面向对象

# 元编程

# 模块与包 

# 网络与Web编程

# 并发程序设计

# 脚本编程与系统管理

# 测试、调试、异常

# C语言拓展